In [335]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [336]:
url = "http://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx"
html_get = urlopen(url).read()
soup_get = BeautifulSoup(html_get, 'html.parser')

In [337]:
viewstate = soup_get.find("input", {"id": "__VIEWSTATE"}).attrs['value']
viewstategenerator = soup_get.find("input", {"id": "__VIEWSTATEGENERATOR"}).attrs['value']

In [338]:
f = open('soup_get.html', 'w')
f.write('%s' % soup_get.prettify())
f.close()

In [339]:
snippet = soup_get.find_all('script')[5]

In [340]:
ctl00_ctl02_HiddenField_encoded = snippet['src'].split('=')[2]

In [341]:
from urllib.parse import unquote
ctl00_ctl02_HiddenField = " ".join(unquote(ctl00_ctl02_HiddenField_encoded).split("+"))

In [342]:
year = 2017
month = 6
date = 6
from urllib.parse import urlencode
postdata = urlencode({
    "ctl00$ctl02": "ctl00$cphMain$ctl00|ctl00$cphMain$btnQuery",
    "ctl00_ctl02_HiddenField": ctl00_ctl02_HiddenField,
    "__EVENTTARGET": "ctl00$cphMain$btnQuery",
    "__EVENTARGUMENT": "",
    "__LASTFOCUS": "",
    "__VIEWSTATE": viewstate,
    "__VIEWSTATEGENERATOR": viewstategenerator,
    "ctl00$cphMain$cboSearch": "防汛重點水庫",
    "ctl00$cphMain$ucDate$cboYear": year,
    "ctl00$cphMain$ucDate$cboMonth": month,
    "ctl00$cphMain$ucDate$cboDay": date,
    "__ASYNCPOST": True
})

In [343]:
postdata = postdata.encode("utf-8")

In [344]:
html_post = urlopen(url, postdata).read()
soup_post = BeautifulSoup(html_post, 'html.parser')

In [345]:
f = open('soup_post.html', 'w')
f.write('%s' % soup_post.prettify())
f.close()

## Parse HTML using tags

### Useful Operation Concepts Classified by Return Data Type
- list: soup.find_all will return a list
- string: soup.prettify() will return string, so use it only on output
- bs4.BeutifulSoup: BeautifulSoup(html, "html.parser", parse_only=...) will return a bs4.BeautifulSoup class,so we can do more bs4 operation on it

In [346]:
from bs4 import SoupStrainer
import os

output_csv_filename = "reservoir.csv"
only_th_tags = SoupStrainer("th")
for idx, element in enumerate(soup_post.find_all(only_th_tags)):
    if idx < 1 or idx > 3:
        if idx < 1:
            with open(output_csv_filename, "w") as f:
                f.write(element.get_text() + " ")
        else:
            with open(output_csv_filename, "a") as f:
                f.write(element.get_text() + " ")
        #print(element.get_text())
        
with open(output_csv_filename, "rb+") as f:
    f.seek(-1, os.SEEK_END)
    f.truncate()
with open(output_csv_filename, "a") as f:            
    f.write("\n")


> 這三欄不計：
> ~~水庫基本數據~~
> ~~每日蓄水統計~~
> ~~即時水情資料~~

CSV 欄位以下開始：
1. 水庫名稱
2. 有效容量(萬立方公尺)
3. 統計時間
4. 集水區降雨量(毫米)
5. 進水量(萬立方公尺)
6. 出水量(萬立方公尺)
7. 與昨日水位差(公尺)
8. 水情時間
9. 水位(公尺)
10. 有效蓄水量(萬立方公尺)
11. 蓄水量百分比(%)

In [347]:

import re
only_tr_tags = SoupStrainer("tr")
only_td_tags = SoupStrainer("td")

for idx, td_details in enumerate(soup_post.find_all(only_tr_tags)):
    if idx > 1 and idx < 22: # reservoir data range from 2 to 21 in the list
        for t_idx, element in enumerate(td_details.find_all(only_td_tags)):
            matchObject = re.match('\S+',element.get_text())
            if matchObject:
                with open(output_csv_filename, "a") as f:
                        f.write(matchObject.group() + " ") 
                #if idx == 2 and t_idx == 0:
                #    with open(output_csv_filename, "w") as f:
                #        f.write(matchObject.group() + " ")
                #else: # if t_idx == 6 should modify timestamp to match ISO861
                #    with open(output_csv_filename, "a") as f:
                #        f.write(matchObject.group() + " ")    
                #print(matchObject.group())
        with open(output_csv_filename, "rb+") as f:
            f.seek(-1, os.SEEK_END)
            f.truncate()
        with open(output_csv_filename, "a") as f:            
            f.write("\n")
